In [36]:
train_pathology_path = 'data/train_pathology.csv'
test_pathology_path = 'data/test_pathology.csv'
train_plants_path = 'data/train_plants.csv'
test_plants_path = 'data/test_plants.csv'

In [37]:
import json
def load_config(config_path = 'config.json'):
    file = open(config_path,'r')
    return json.load(file)

config = load_config()

In [38]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import tensorflow as tf, tensorflow.keras.backend as K
import matplotlib.pyplot as plt

from os import mkdir, listdir
from os.path import exists, isfile, join
import json

from tensorflow.keras.layers import Dense,BatchNormalization,Dropout,Input,Flatten,Activation,Conv2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.utils.class_weight import compute_class_weight

In [39]:
tf.config.experimental.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [40]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

In [41]:
def train_val_split(x_train,y_train, test_size = 0.2):
    x_train = np.array(x_train)
    y_train = np.array(y_train)
    sss = StratifiedShuffleSplit(n_splits=1, test_size = test_size, random_state=0)
    for train_index, test_index in sss.split(x_train, y_train):
        x_train, x_test = x_train[train_index], x_train[test_index]
        y_train, y_test = y_train[train_index], y_train[test_index]
    return x_train, y_train, x_test, y_test

In [42]:
def get_train_data(path):
    
    df = pd.read_csv(path)
    
    paths = df.pop('path')
    
    paths = [str(elem) for elem in paths]
    
    y = df.to_numpy().astype('float32')
 
    return np.array(paths),y

In [43]:
x_train_pathology,y_train_pathology = get_train_data(train_pathology_path)
x_train_plants,y_train_plants = get_train_data(train_plants_path)

In [44]:
x_train_pathology,y_train_pathology,x_val_pathology,y_val_pathology = train_val_split(x_train_pathology,y_train_pathology)

x_train_plants,y_train_plants,x_val_plants,y_val_plants = train_val_split(x_train_plants,y_train_plants)

In [45]:
from tensorflow.keras.models import model_from_json

def load_model(model_path,weights_path):
    
    json_file = open(model_path, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    loaded_model.load_weights(weights_path)

    print("model loaded successfully")
    return loaded_model


In [46]:
img_height = config['img_height']
img_width = config['img_width']
epochs = config['epochs']
batch_size = config['batch_size']

need_training = config['need_training']

model_name = config['model_name']
model_json_path = config['model_json_path']
model_weights_path = config['model_weights_path']

nb_classes = y_train_pathology.shape[1]

In [47]:
import efficientnet.tfkeras as efn

def get_model():
    '''
    inp = Input(shape=(img_height,img_width,3))
    x = Flatten()(inp)
    x = Dense(nb_classes,activation='softmax')(x)
    return Model(inp,x)
    
    '''
    
    base_model = efn.EfficientNetB0(weights='imagenet',
                              include_top=False,
                              input_shape=(img_height,img_width, 3),
                              pooling='avg')
        
    x = base_model.output
    predictions = Dense(nb_classes,activation='softmax')(x)
    
    return Model(base_model.input,predictions)
    

In [48]:
model = None
if (model_json_path is None):
    model = get_model()
else:
    model = load_model(model_json_path,model_weights_path)

In [49]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 128, 192, 3) 0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 64, 96, 32)   864         input_2[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 64, 96, 32)   128         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 64, 96, 32)   0           stem_bn[0][0]                    
____________________________________________________________________________________________

In [50]:
#preparing metrics

if not exists(join('output',model_name)):
    mkdir(join('output',model_name))

save_model_path = join('output',model_name,'model_weights.h5')

MODEL_CFG = {
    'optimizer': Adam(lr=0.001),
    'loss': 'categorical_crossentropy',
    'metrics': ['accuracy'],
    'name': save_model_path
}

CL_BEST_MODEL= ModelCheckpoint(MODEL_CFG['name'], 
                                 monitor='val_loss', 
                                 verbose=1, 
                                 save_best_only=True,
                                 save_weights_only=True,
                                 mode='min')
    
CL_REDUCE_LR = ReduceLROnPlateau(monitor='val_loss',
                                  factor=0.5,
                                  verbose=0,
                                  min_lr = 1e-5,
                                  patience=10)

CL_EARLY_STOPPING = EarlyStopping(monitor = "val_loss" , verbose = 1 , mode = 'min' , patience = 50)
CALLBACKS = [CL_BEST_MODEL,CL_REDUCE_LR,CL_EARLY_STOPPING]

In [51]:
model.compile(
            optimizer = MODEL_CFG['optimizer'],
            loss = MODEL_CFG['loss'],
            metrics = MODEL_CFG['metrics'],
        )

In [52]:
def train_model(model,
                train_generator,
                val_generator = None,
                epochs = 10,
                steps = 22,
                class_weights = None
               ):

    history = model.fit_generator(
        generator = train_generator,
        epochs=epochs,
        verbose=1,
        validation_data = val_generator,
        callbacks=CALLBACKS,
        class_weight = class_weights
    )
    
    return history

In [53]:
from DataGenerator import DataGenerator

gen_train = DataGenerator(x_train_pathology,y_train_pathology,batch_size,augment = config['do_augmentation'])
gen_val = DataGenerator(x_val_pathology,y_val_pathology,batch_size,shuffle=False,augment=False)

In [54]:
history = None
if need_training:
    history = train_model(model,gen_train,gen_val,epochs=epochs)

/opt/anaconda/lib/python3.7/site-packages/imgaug/augmenters/segmentation.py:272: FutureWarning: skimage.measure.label's indexing starts from 0. In future version it will start from 1. To disable this warning, explicitely set the `start_label` parameter to 1.
  image, n_segments=n_segments_samples[i], compactness=10)


Epoch 1/2
69/69 [==============================] - ETA: 0s - loss: 0.7690 - accuracy: 0.6712
Epoch 00001: val_loss improved from inf to 0.45329, saving model to output/PSU_model/model_weights.h5
69/69 [==============================] - 92s 1s/step - loss: 0.7690 - accuracy: 0.6712 - val_loss: 0.4533 - val_accuracy: 0.8493 - lr: 0.0010
Epoch 2/2
26/69 [==========>...................] - ETA: 51s - loss: 0.6412 - accuracy: 0.7404

KeyboardInterrupt: 

In [ ]:
import pickle

if history is not None:
    history_path = join('output',model_name,'history')
    with open(history_path, 'wb') as f:
        pickle.dump(history.history, f)

In [24]:
model_json = model.to_json()
with open(join('output',model_name,'model.json'), "w") as json_file:
    json_file.write(model_json)